# Record

This article is my attempt to explain how the utility [Record](https://www.typescriptlang.org/docs/handbook/utility-types.html#recordkeys-type) work:


Seen from the [source](https://github.com/microsoft/TypeScript/blob/2f34e57ed39ba67b4c4b26bbcb6088cf39aa308a/src/lib/es5.d.ts#L1581), here is the implementation of Record:

In [30]:
{
    type Record<K extends keyof any, T> = {
        [P in K]: T;
    };
}

In the generic parameter, the operator `keyof` returns a union of all types for the keys in the given operand ([Ref](https://www.typescriptlang.org/docs/handbook/2/keyof-types.html)). 

In this case, the operand is `any` type, and the return of the operation is `string | number | symbol`. 

In [31]:
import {IsEqual} from 'type-fest'
{
    type KeyOfAny = keyof any
    const expect1:IsEqual<KeyOfAny, string | number | symbol> = true 
}

The type parameter:

```
<K extends keyof any>
```
can therefore be rewritten as:

```
<K extends keyof string | number | symbol >
```
The above expression constraints `K` to one of `string`, `number` or `symbol`:

In [32]:
import {IsEqual} from 'type-fest'  
{
    type IsKeyOfAny<K> = K extends keyof any ? true : false

    const expect1: IsEqual<IsKeyOfAny<string>, true> = true 
    const expect2: IsEqual<IsKeyOfAny<number>, true> = true 
    const expect3: IsEqual<IsKeyOfAny<symbol>, true> = true 

    const expect4: IsEqual<IsKeyOfAny<boolean>, false> = true 
}

In the context of javascript object, `keyof any` represent the types for key that can be use as an object's property. These types are `string`, `number` or `symbol`.  

Since we can describe an object using the [index signatures syntax](https://www.typescriptlang.org/docs/handbook/2/objects.html#index-signatures), we can write the object type description for each of this three cases:

In [33]:
// For the case when K is string:
{
    type MyObject<T> = {
        [P: string]: T 
    } 
    const o: MyObject<string> = {}
    o['1'] = 'some string'
}

// For the case when K is number:
{
    type MyObject<T> = {
        [P: number]: T 
    } 
    const o: MyObject<string> = {}
    o[1] = 'some string'
}

// For the case when K is symbol:
{
    type MyObject2<T> = {
        [P: symbol]: T 
    } 
    const o: MyObject2<string> = {}
    o[Symbol(1)] = 'some string'
}


To avoid repetition we can utilize the constraints `K extends string | number | symbol`, or `K extends keyof any`; however, this will cause compiler error:

In [38]:
{
    type MyObject<K extends keyof any, T> = {
        [P: K]: T 
    //   ^--- Error: An index signature parameter type cannot be a literal type or generic type. 
    //               Consider using a mapped object type instead.
    } 

    // For the case when K is string:
    {
        const o: MyObject<string, string> = {}
        o['1'] = 'some string'
        console.log(o)
    }

    // For the case when K is number:
    {
        const o: MyObject<number, string> = {}
        o[1] = 'some string'
        console.log(o)
    }

    // For the case when K is symbol:
    {
        const o: MyObject<symbol, string> = {}
        o[Symbol(1)] = 'some string'
        console.log(o)
    }
}

3:10 - An index signature parameter type cannot be a literal type or generic type. Consider using a mapped object type instead.


The typescript compiler does not let you use generic parameter (`K`) as a type of index (`P`). It suggest you to use a [mapped object type](https://github.com/Microsoft/TypeScript/pull/12114): 

In [37]:
{
    type MyObject<K extends keyof any, T> = {
        [P in K]: T 
    } 

    // For the case when K is string:
    {
        const o: MyObject<string, string> = {}
        o['1'] = 'some string'
        console.log(o)
    }

    // For the case when K is number:
    {
        const o: MyObject<number, string> = {}
        o[1] = 'some string'
        console.log(o)
    }

    // For the case when K is symbol:
    {
        const o: MyObject<symbol, string> = {}
        o[Symbol(1)] = 'some string'
        console.log(o)
    }
}

{ '1': 'some string' }
{ '1': 'some string' }
{ [Symbol(1)]: 'some string' }


 In the above I have named the type description for the object `MyObject`, which is actually the `Record` utility in typescript.   

Note that for an javascript object, key as type `number` and type `string` are treated as a same key of record:

In [ ]:
{
    const foo = {
        '1': 'string',
        1: 'number'
    //  ^--- Error: Duplicate identifier '1' 
    }
}